In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat 
import os
import glob

import pandas as pd
from tqdm.notebook import tqdm

In [2]:
training_methods = ["dino", "supervised"] 
depth_list = [4, 8, 12]
num_upsample = 30
num_models = 6
num_heads = 6

In [3]:
dist_mat_dict = {}

In [4]:
# load subject gaze pos
subj_gaze_pos = np.load("../preprocessed_data/goodsubj_gaze_pos.npz", allow_pickle=True)["gaze_data_goodsubj"]
num_subj = len(subj_gaze_pos)

# load vit gaze pos
vit_gaze_pos_data = np.load("../preprocessed_data/vit_gaze_pos_upsample.npz", allow_pickle=True)

In [5]:
for tm in tqdm(training_methods):
    dist_mat_dict[tm] = {}
    vit_gaze_pos_data_tm = vit_gaze_pos_data[tm].item()
    for depth in tqdm(depth_list):
        vit_gaze_pos_data_depth = vit_gaze_pos_data_tm[str(depth)]

        dist_mat = np.zeros((num_subj, num_models, depth, num_heads+1))
        for i in tqdm(range(num_subj)):
            gaze_diff = vit_gaze_pos_data_depth - subj_gaze_pos[i]
            diff_norm = np.linalg.norm(gaze_diff, axis=-1)
            dist_mat[i] = np.nanmedian(diff_norm, axis=-1)
        dist_mat_dict[tm][str(depth)] = dist_mat

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

In [6]:
np.savez_compressed("../preprocessed_data/subj2vit_dist.npz", **dist_mat_dict)